In [6]:
import instructor
import litellm
from pydantic import BaseModel


class User(BaseModel):
    name: str
    age: int


def track_cost_callback(kwargs, completion_response, start_time, end_time):
    try:
        response_cost = kwargs.get("response_cost", 0)
        print(f"Cost for {kwargs['model']}: ${response_cost:.6f}")
        duration = end_time - start_time
        print(f"Duration: {duration.total_seconds():.2f} seconds")
        print(f"Response: {completion_response}")
        if isinstance(completion_response, litellm.ModelResponse) and 'usage' in completion_response:
            usage = completion_response['usage']
            print(f"Completion tokens: {usage.completion_tokens}")
            print(f"Prompt tokens: {usage.prompt_tokens}")
            print(f"Total tokens: {usage.total_tokens}")
    except:
        pass

litellm.success_callback = [track_cost_callback]

client = instructor.from_litellm(litellm.completion)

response = client.chat.completions.create(
    model="gpt-4o-mini",
    max_tokens=1024,
    messages=[
        {
            "role": "user",
            "content": "Extract Jason is 25 years old.",
        }
    ],
    response_model=User,
)

print(response)

Cost for gpt-4o-mini: $0.000017name='Jason' age=25

Duration: 0.75 seconds
Response: ModelResponse(id='chatcmpl-9msNKIgOjLK1nYz0PoRnzoQ5X3elw', choices=[Choices(finish_reason='stop', index=0, message=Message(content=None, role='assistant', tool_calls=[ChatCompletionMessageToolCall(function=Function(arguments='{"name":"Jason","age":25}', name='User'), id='call_DlNAW1x0Yt3fgAi0So4yQVsX', type='function')]))], created=1721435790, model='gpt-4o-mini-2024-07-18', object='chat.completion', system_fingerprint='fp_661538dc1f', usage=Usage(completion_tokens=9, prompt_tokens=76, total_tokens=85))
Completion tokens: 9
Prompt tokens: 76
Total tokens: 85


In [8]:
import litellm

with open('test_page.html', 'r') as file:
    html_content = file.read()

token_count = litellm.token_counter(model="gpt-4o-mini", text=html_content)
print(f"Token count: {token_count}")

from bs4 import BeautifulSoup
from pydantic import BaseModel
from typing import List, Optional

class JobDetails(BaseModel):
    title: str
    company: Optional[str]
    location: Optional[str]
    responsibilities: List[str]
    application_link: Optional[str]
    salary_range: Optional[str]
    qualifications: List[str]
    company_details: Optional[str]

# Use gpt-4o-mini to extract job details
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {
            "role": "user",
            "content": f"Extract the following job details from this HTML content:\n"
                       f"- Job title\n"
                       f"- Company name\n"
                       f"- Location\n"
                       f"- List of responsibilities\n"
                       f"- Application link\n"
                       f"- Salary range (if available)\n"
                       f"- List of qualifications\n"
                       f"- Company details or description\n\n"
                       f"HTML content:\n{html_content}",
        }
    ],
    response_model=JobDetails,
)

print("Extracted Job Details:")
import json
print(json.dumps(response.dict(), indent=2))



Token count: 28787
Cost for gpt-4o-mini: $0.004627Extracted Job Details:
{
  "title": "Software Engineer (Remote)",
  "company": "Seek One Productions",
  "location": "Houston, TX, United States",
  "responsibilities": [
    "Collaborate with senior engineers to design, develop, and maintain software applications",
    "Write clean, efficient, and maintainable code in various programming languages",
    "Participate in code reviews to ensure code quality and share knowledge",
    "Debug and troubleshoot software issues to ensure optimal performance",
    "Implement new features and enhancements based on project requirements",
    "Contribute to the development and maintenance of technical documentation",
    "Assist in testing and quality assurance processes",
    "Stay current with emerging technologies and industry trends",
    "Participate in agile development processes and sprints",
    "Provide support and assistance to team members as needed",
    "Ensure software is scalable, re